In [1]:
import cv2
import numpy as np
import os
import keras
import Preparation
import tensorflow as tf
import time

from keras.layers.core import Dense
from keras.models import load_model

In [2]:
 # prepare dataset
X_test, Y_test = Preparation.test_train_datasets(r'D:\PycharmProjects\smiling_detection\datasets\test_folder')
X_train, Y_train = Preparation.test_train_datasets(r'D:\PycharmProjects\smiling_detection\datasets\train_folder')

X_train, Y_train = Preparation.reformation(X_train, Y_train)
X_test, Y_test = Preparation.reformation(X_test, Y_test)

In [3]:
X_train.shape

(2800, 227, 227, 3)

In [4]:
root_logdir = os.path.join(os.curdir, "logs\\fit\\")


def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)


run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [5]:
# creating AlexNet CNN
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(227, 227, 3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(2, activation='softmax')
])


In [6]:
model.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.SGD(learning_rate=0.001),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 batch_normalization (BatchN  (None, 55, 55, 96)       384       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 batch_normalization_1 (Batc  (None, 27, 27, 256)      1024      
 hNormalization)                                                 
                                                        

In [7]:
model.fit(X_train,
          Y_train,
          epochs=50,
          validation_split=0.2,
          callbacks=[tensorboard_cb],
          verbose=1)
score = model.evaluate(X_test, Y_test, verbose=1)
print("Test score:", score[0])
print('Test accuracy:', score[1])

Epoch 1/50
70/70 [==============================] - 87s 1s/step - loss: 1.4863 - accuracy: 0.5638 - val_loss: 0.6857 - val_accuracy: 0.6161
Epoch 2/50
70/70 [==============================] - 89s 1s/step - loss: 1.0777 - accuracy: 0.6228 - val_loss: 0.6947 - val_accuracy: 0.5054
Epoch 3/50
70/70 [==============================] - 84s 1s/step - loss: 0.9819 - accuracy: 0.6317 - val_loss: 0.6464 - val_accuracy: 0.6464
Epoch 4/50
70/70 [==============================] - 87s 1s/step - loss: 0.7627 - accuracy: 0.6951 - val_loss: 0.6365 - val_accuracy: 0.6482
Epoch 5/50
70/70 [==============================] - 84s 1s/step - loss: 0.7305 - accuracy: 0.7018 - val_loss: 0.5985 - val_accuracy: 0.6661
Epoch 6/50
70/70 [==============================] - 85s 1s/step - loss: 0.6737 - accuracy: 0.7170 - val_loss: 0.5837 - val_accuracy: 0.7071
Epoch 7/50
70/70 [==============================] - 83s 1s/step - loss: 0.6005 - accuracy: 0.7487 - val_loss: 0.9857 - val_accuracy: 0.5375
Epoch 8/50
70/70 [==

In [8]:
model.save('model')

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


In [3]:
loaded_model = load_model('model')

In [6]:
# print predict
def prediction(model, path: str):
    image_arr = Preparation.get_array_from_image(path)
    reform_arr = image_arr / 255
    pred = model.predict(reform_arr)
    gray_image = cv2.cvtColor(image_arr[0], cv2.COLOR_BGR2GRAY)
    mouth_cascade = cv2.CascadeClassifier(
        r'D:\PycharmProjects\smiling_detection\haarcascade_mcs_mouth.xml')
    mouth = mouth_cascade.detectMultiScale(gray_image, 1.7, 20)
    for (x_smile, y_smile, w_smile, h_smile) in mouth:
        cv2.rectangle(image_arr[0], (x_smile, y_smile), (x_smile + w_smile, y_smile + h_smile), (255, 0, 130), 2)
    font = cv2.FONT_HERSHEY_SIMPLEX
    if pred[0][0] == 1:
        cv2.putText(image_arr[0], 'Non_smile', (50, 50), font, 1.2, (255, 255, 255), 2)
    else:
        cv2.putText(image_arr[0], 'Smile', (50, 50), font, 1.2, (255, 255, 255), 2)
    cv2.imshow('', image_arr[0])
    cv2.waitKey(200)


prediction(loaded_model, r'D:\PycharmProjects\smiling_detection\1.jpg')